# Automate Jira ↔ GitHub with `codex-cli`

## What you'll build

By the end of this walkthrough, you'll have an end-to-end automation that turns a **labelled Jira issue** into a
fully-fledged **GitHub pull request**, keeps the Jira ticket updated throughout the review process, and finally merges
the code once checks pass – all driven by the
[`codex-cli`](https://github.com/openai/openai-codex) agent working inside a GitHub Action.

# <img src="./images/codex_action.png" alt="Full data-flow diagram" width="500"/>


The flow is:
1. Label a Jira issue 
2. Jira Automation calls the GitHub Action 
3. The action spins up `codex-cli` to implement the change 
4. A PR is opened
5. Jira is transitioned & annotated – creating a neat, zero-click loop

## Prerequisites

* Jira: project admin rights + ability to create automation rules  
* GitHub: write access, permission to add repository secrets, and a protected `main` branch  
* API keys & secrets placed as repository secrets:
  * `OPENAI_API_KEY` – your OpenAI key for `codex-cli`  
  * `JIRA_BASE_URL`, `JIRA_EMAIL`, `JIRA_API_TOKEN` – for REST calls from the action  
* `codex-cli` installed locally (`pnpm add -g @openai/codex`) for ad-hoc testing  
* A repository that already contains a `.github/workflows/` folder (any Node project is fine)


## Create the Jira Automation Rule

<img src="./images/jira_rule.png" alt="Automation Rule" width="500"/>

* **Trigger – Issue updated**: fires whenever labels change  
* **Condition – `labels` contains `autocodex`**: limits noise to explicitly opted-in issues  
* **Action – Send web request**: POSTs the issue key/summary/description to your GitHub Action's `workflow_dispatch` endpoint


## Add the GitHub Action

Create a workflow file in your `.github/workflows/` directory with the following YAML (Modify it as needed):


```yaml
name: Codex Automated PR
on:
  workflow_dispatch:
    inputs:
      issue_key:
        description: 'JIRA issue key (e.g., PROJ-123)'
        required: true
      issue_summary:
        description: 'Brief summary of the issue'
        required: true
      issue_description:
        description: 'Detailed issue description'
        required: true

permissions:
  contents: write           # allow the action to push code & open the PR
  pull-requests: write      # allow the action to create and update PRs

jobs:
  codex_auto_pr:
    runs-on: ubuntu-latest

    steps:
    # 0 – Checkout repository
    - uses: actions/checkout@v4
      with:
        fetch-depth: 0       # full history → lets Codex run tests / git blame if needed

    # 1 – Set up Node.js and Codex
    - uses: actions/setup-node@v4
      with:
        node-version: 22
    - run: pnpm add -g @openai/codex

    # 2 – Export / clean inputs (available via $GITHUB_ENV)
    - id: vars
      run: |
        echo "ISSUE_KEY=${{ github.event.inputs.issue_key }}"        >> $GITHUB_ENV
        echo "TITLE=${{ github.event.inputs.issue_summary }}"        >> $GITHUB_ENV
        echo "RAW_DESC=${{ github.event.inputs.issue_description }}" >> $GITHUB_ENV
        DESC_CLEANED=$(echo "${{ github.event.inputs.issue_description }}" | tr '\n' ' ' | sed 's/"/'\''/g')
        echo "DESC=$DESC_CLEANED"                                    >> $GITHUB_ENV
        echo "BRANCH=codex/${{ github.event.inputs.issue_key }}"     >> $GITHUB_ENV

    # 3 – Transition Jira issue to "In Progress"
    - name: Jira – Transition to In Progress
      env:
        ISSUE_KEY:      ${{ env.ISSUE_KEY }}
        JIRA_BASE_URL:  ${{ secrets.JIRA_BASE_URL }}
        JIRA_EMAIL:     ${{ secrets.JIRA_EMAIL }}
        JIRA_API_TOKEN: ${{ secrets.JIRA_API_TOKEN }}
      run: |
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/transitions" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  '{"transition":{"id":"21"}}'

    # 4 – Set Git author for CI commits
    - run: |
        git config user.email "github-actions[bot]@users.noreply.github.com"
        git config user.name  "github-actions[bot]"

    # 5 – Let Codex implement & commit (no push yet)
    - name: Codex implement & commit
      env:
        OPENAI_API_KEY:  ${{ secrets.OPENAI_API_KEY }}
        CODEX_QUIET_MODE: "1"          # suppress chatty logs
      run: |
        set -e
        codex --approval-mode full-auto --no-terminal --quiet \
              "Implement JIRA ticket $ISSUE_KEY: $TITLE. $DESC"

        git add -A
        git commit -m "feat($ISSUE_KEY): $TITLE"

    # 6 – Open (and push) the PR in one go
    - id: cpr
      uses: peter-evans/create-pull-request@v6
      with:
        token:  ${{ secrets.GITHUB_TOKEN }}
        base:   main
        branch: ${{ env.BRANCH }}
        title:  "${{ env.TITLE }} (${{ env.ISSUE_KEY }})"
        body: |
          Auto-generated by Codex for JIRA **${{ env.ISSUE_KEY }}**.
          ---
          ${{ env.DESC }}

    # 7 – Transition Jira to "In Review" & drop the PR link
    - name: Jira – Transition to In Review & Comment PR link
      env:
        ISSUE_KEY:      ${{ env.ISSUE_KEY }}
        JIRA_BASE_URL:  ${{ secrets.JIRA_BASE_URL }}
        JIRA_EMAIL:     ${{ secrets.JIRA_EMAIL }}
        JIRA_API_TOKEN: ${{ secrets.JIRA_API_TOKEN }}
        PR_URL:         ${{ steps.cpr.outputs.pull-request-url }}
      run: |
        # Status transition
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/transitions" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  '{"transition":{"id":"31"}}'

        # Comment with PR link
        curl -sS -X POST \
          --url   "$JIRA_BASE_URL/rest/api/3/issue/$ISSUE_KEY/comment" \
          --user  "$JIRA_EMAIL:$JIRA_API_TOKEN" \
          --header 'Content-Type: application/json' \
          --data  "{\"body\":{\"type\":\"doc\",\"version\":1,\"content\":[{\"type\":\"paragraph\",\"content\":[{\"type\":\"text\",\"text\":\"PR created: $PR_URL\"}]}]}}"
```


## Key Steps in the Workflow

1. **Codex Implementation & Commit** (Step 5)
   - Uses OpenAI API to implement the JIRA ticket requirements
   - Runs codex CLI in full-auto mode without terminal interaction
   - Commits all changes with standardized commit message

2. **Create Pull Request** (Step 6) 
   - Uses peter-evans/create-pull-request action
   - Creates PR against main branch
   - Sets PR title and description from JIRA ticket info
   - Returns PR URL for later use

3. **JIRA Updates** (Step 7)
   - Transitions ticket to "In Review" status via JIRA API
   - Posts comment with PR URL on the JIRA ticket
   - Uses curl commands to interact with JIRA REST API


## Label an Issue

Attach the special `autocodex` label to any bug/feature ticket:

1. **During creation** – add it in the "Labels" field before hitting *Create*  
2. **Existing issue** – hover the label area → click the pencil icon → type `autocodex`

<img src="./images/add_label.png" alt="Adding a label" width="500"/>


## End-to-end Flow in Action

1. Jira label added → Automation triggers
2. `workflow_dispatch` fires; action spins up on GitHub
3. `codex-cli` edits the codebase & commits
4. PR is opened on the generated branch
5. Jira is moved to **In Review** and a comment with the PR URL is posted
6. Reviewers are notified per your normal branch protection settings

<img src="./images/jira_comment.png" alt="Jira comment with PR link" width="300"/>
<img src="./images/jira_status_change.png" alt="Jira status transition to In Review" width="300"/>


## Review & Merge the PR

* Confirm CI passes – tests, lint, build
* Approve & squash-merge
* The default branch protection will auto-close the Jira ticket once the PR merges if your *Smart Commits* integration is enabled

## Conclusion

This automation streamlines your development workflow by creating a seamless integration between Jira and GitHub:

* **Zero-click implementation** - AI handles the code changes based on ticket descriptions
* **Automatic status tracking** - Tickets progress through your workflow without manual updates
* **Improved developer experience** - Focus on reviewing quality code instead of writing boilerplate
* **Reduced handoff friction** - The PR is ready for review as soon as the ticket is labeled

The `codex-cli` tool is a powerful AI coding assistant that automates repetitive programming tasks. You can explore more about it at:

* [OpenAI Codex-CLI GitHub Repository](https://github.com/openai/codex/)